In [8]:
import pandas as pd
import re
from langdetect import detect


In [3]:
sf_reviews = pd.read_csv('sfReviews.csv')
sf_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285275 entries, 0 to 285274
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     285275 non-null  int64 
 1   id             285275 non-null  int64 
 2   date           285275 non-null  object
 3   reviewer_id    285275 non-null  int64 
 4   reviewer_name  285275 non-null  object
 5   comments       285131 non-null  object
dtypes: int64(3), object(3)
memory usage: 13.1+ MB


In [4]:
sf_reviews.dropna(inplace=True)
sf_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 285131 entries, 0 to 285274
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     285131 non-null  int64 
 1   id             285131 non-null  int64 
 2   date           285131 non-null  object
 3   reviewer_id    285131 non-null  int64 
 4   reviewer_name  285131 non-null  object
 5   comments       285131 non-null  object
dtypes: int64(3), object(3)
memory usage: 15.2+ MB


In [5]:
sf_reviews['date'] = pd.to_datetime(sf_reviews['date'])
sf_reviews['year'] = sf_reviews.date.dt.year
print(sf_reviews.info())
sf_reviews.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 285131 entries, 0 to 285274
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   listing_id     285131 non-null  int64         
 1   id             285131 non-null  int64         
 2   date           285131 non-null  datetime64[ns]
 3   reviewer_id    285131 non-null  int64         
 4   reviewer_name  285131 non-null  object        
 5   comments       285131 non-null  object        
 6   year           285131 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 17.4+ MB
None


,listing_id,id,date,reviewer_id,reviewer_name,comments,year
0,958,5977,2009-07-23,15695,Edmund C,"Our experience was, without a doubt, a five st...",2009
1,958,6660,2009-08-03,26145,Simon,Returning to San Francisco is a rejuvenating t...,2009
2,958,11519,2009-09-27,25839,Denis,We were very pleased with the accommodations a...,2009
3,958,16282,2009-11-05,33750,Anna,We highly recommend this accomodation and agre...,2009
4,958,26008,2010-02-13,15416,Venetia,Holly's place was great. It was exactly what I...,2010


In [6]:
sf_reviews.year.value_counts()

2019    62327
2018    49701
2021    42906
2017    38666
2016    27716
2020    25242
2015    17795
2014    11444
2013     5224
2012     2467
2011     1253
2010      351
2009       39
Name: year, dtype: int64

In [9]:
#Pre-Processing
sf_reviews["comments_adj"]= sf_reviews["comments"].apply(lambda x:re.sub("[^A-Za-z]", " ", x.strip()))

## Lower Case 
#df['reviews_str'] = df['reviews_str'].apply(lambda x: " ".join(x.lower() for x in x.split()))

## Numbers Remove
sf_reviews['comments_adj'] = sf_reviews['comments'].str.replace('[\d]', '')

## Puncations Remove
#df['reviews_str'] = df['reviews_str'].str.replace('[^\w\s]','')

# remove \n 
sf_reviews['comments_adj'] = sf_reviews['comments'].str.replace('\n', ' ')

sf_reviews

/var/folders/71/xlbnyb5s341dqzxgjd0pzzqw0000gn/T/ipykernel_4294/867753885.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  sf_reviews['comments_adj'] = sf_reviews['comments'].str.replace('[\d]', '')


,listing_id,id,date,reviewer_id,reviewer_name,comments,year,comments_adj
0,958,5977,2009-07-23,15695,Edmund C,"Our experience was, without a doubt, a five st...",2009,"Our experience was, without a doubt, a five st..."
1,958,6660,2009-08-03,26145,Simon,Returning to San Francisco is a rejuvenating t...,2009,Returning to San Francisco is a rejuvenating t...
2,958,11519,2009-09-27,25839,Denis,We were very pleased with the accommodations a...,2009,We were very pleased with the accommodations a...
3,958,16282,2009-11-05,33750,Anna,We highly recommend this accomodation and agre...,2009,We highly recommend this accomodation and agre...
4,958,26008,2010-02-13,15416,Venetia,Holly's place was great. It was exactly what I...,2010,Holly's place was great. It was exactly what I...
...,...,...,...,...,...,...,...,...
285270,53258020,501567230864641900,2021-11-22,104345330,Paige,We had a lovely stay at Craigs place. The room...,2021,We had a lovely stay at Craigs place. The room...
285271,53258020,503009892202171652,2021-11-24,203525522,Martha,"Craig is an exceptional host, and his rooms an...",2021,"Craig is an exceptional host, and his rooms an..."
285272,53378110,500100974673293551,2021-11-20,3044891,David,We've rented the entire home (upstairs and dow...,2021,We've rented the entire home (upstairs and dow...
285273,53405558,507272106926274471,2021-11-30,306384286,Juan,One of the best airbnb's in San Francisco ! wo...,2021,One of the best airbnb's in San Francisco ! wo...


In [10]:
# keep only reviews in English
def non_eng(x):
    try:
        if detect(x) != 'en':
            return 'Not English'
    except:
        pass

sf_reviews['is_english'] = sf_reviews.comments_adj.apply(non_eng)

In [11]:
sf_reviews.is_english.value_counts()

Not English    16737
Name: is_english, dtype: int64

In [17]:
sf_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,year,comments_adj,is_english
0,958,5977,2009-07-23,15695,Edmund C,"Our experience was, without a doubt, a five st...",2009,"Our experience was, without a doubt, a five st...",None
1,958,6660,2009-08-03,26145,Simon,Returning to San Francisco is a rejuvenating t...,2009,Returning to San Francisco is a rejuvenating t...,None
2,958,11519,2009-09-27,25839,Denis,We were very pleased with the accommodations a...,2009,We were very pleased with the accommodations a...,None
3,958,16282,2009-11-05,33750,Anna,We highly recommend this accomodation and agre...,2009,We highly recommend this accomodation and agre...,None
4,958,26008,2010-02-13,15416,Venetia,Holly's place was great. It was exactly what I...,2010,Holly's place was great. It was exactly what I...,None


In [18]:
sf_reviews.shape

(285131, 9)

In [21]:
sf_reviews['city'] = 'SF'
sf_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,year,comments_adj,is_english,city
0,958,5977,2009-07-23,15695,Edmund C,"Our experience was, without a doubt, a five st...",2009,"Our experience was, without a doubt, a five st...",None,SF
1,958,6660,2009-08-03,26145,Simon,Returning to San Francisco is a rejuvenating t...,2009,Returning to San Francisco is a rejuvenating t...,None,SF
2,958,11519,2009-09-27,25839,Denis,We were very pleased with the accommodations a...,2009,We were very pleased with the accommodations a...,None,SF
3,958,16282,2009-11-05,33750,Anna,We highly recommend this accomodation and agre...,2009,We highly recommend this accomodation and agre...,None,SF
4,958,26008,2010-02-13,15416,Venetia,Holly's place was great. It was exactly what I...,2010,Holly's place was great. It was exactly what I...,None,SF


In [22]:
# create a new dataframe with only reviews that in English
new_df = sf_reviews[sf_reviews['is_english'] != 'Not English']
new_df.shape

(268394, 10)

## Los Angeles 

In [12]:
la_reviews = pd.read_csv('laReviews.csv')
la_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1144105 entries, 0 to 1144104
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   listing_id     1144105 non-null  int64 
 1   id             1144105 non-null  int64 
 2   date           1144105 non-null  object
 3   reviewer_id    1144105 non-null  int64 
 4   reviewer_name  1144103 non-null  object
 5   comments       1142997 non-null  object
dtypes: int64(3), object(3)
memory usage: 52.4+ MB


In [13]:
la_reviews.dropna(inplace=True)
la_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1142995 entries, 0 to 1144104
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   listing_id     1142995 non-null  int64 
 1   id             1142995 non-null  int64 
 2   date           1142995 non-null  object
 3   reviewer_id    1142995 non-null  int64 
 4   reviewer_name  1142995 non-null  object
 5   comments       1142995 non-null  object
dtypes: int64(3), object(3)
memory usage: 61.0+ MB


In [14]:
la_reviews['date'] = pd.to_datetime(la_reviews['date'])
la_reviews['year'] = la_reviews.date.dt.year
print(la_reviews.info())
la_reviews.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1142995 entries, 0 to 1144104
Data columns (total 7 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   listing_id     1142995 non-null  int64         
 1   id             1142995 non-null  int64         
 2   date           1142995 non-null  datetime64[ns]
 3   reviewer_id    1142995 non-null  int64         
 4   reviewer_name  1142995 non-null  object        
 5   comments       1142995 non-null  object        
 6   year           1142995 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 69.8+ MB
None


,listing_id,id,date,reviewer_id,reviewer_name,comments,year
0,109,449036,2011-08-15,927861,Edwin,The host canceled my reservation the day befor...,2011
1,109,74506539,2016-05-15,22509885,Jenn,Me and two friends stayed for four and a half ...,2016
2,2708,13994902,2014-06-09,10905424,Kuberan,i had a wonderful stay. Everything from start ...,2014
3,2708,14606598,2014-06-23,2247288,Camilla,Charles is just amazing and he made my stay sp...,2014
4,2708,39597339,2015-07-25,27974696,Fallon,Staying with Chas was an absolute pleasure. He...,2015


In [15]:
la_reviews.year.value_counts()

2019    267501
2021    245948
2018    189776
2020    168449
2017    123282
2016     77687
2015     40051
2014     18319
2013      7483
2012      3054
2011      1218
2010       211
2009        16
Name: year, dtype: int64

In [20]:
#Pre-Processing
la_reviews["comments_adj"]= la_reviews["comments"].apply(lambda x:re.sub("[^A-Za-z]", " ", x.strip()))

## Lower Case 
#df['reviews_str'] = df['reviews_str'].apply(lambda x: " ".join(x.lower() for x in x.split()))

## Numbers Remove
la_reviews['comments_adj'] = la_reviews['comments'].str.replace('[\d]', '')

## Puncations Remove
#df['reviews_str'] = df['reviews_str'].str.replace('[^\w\s]','')

# remove \n 
la_reviews['comments_adj'] = la_reviews['comments'].str.replace('\n', ' ')

la_reviews

/var/folders/71/xlbnyb5s341dqzxgjd0pzzqw0000gn/T/ipykernel_4294/24514923.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  la_reviews['comments_adj'] = la_reviews['comments'].str.replace('[\d]', '')


,listing_id,id,date,reviewer_id,reviewer_name,comments,year,is_english,comments_adj
0,109,449036,2011-08-15,927861,Edwin,The host canceled my reservation the day befor...,2011,None,The host canceled my reservation the day befor...
1,109,74506539,2016-05-15,22509885,Jenn,Me and two friends stayed for four and a half ...,2016,None,Me and two friends stayed for four and a half ...
2,2708,13994902,2014-06-09,10905424,Kuberan,i had a wonderful stay. Everything from start ...,2014,None,i had a wonderful stay. Everything from start ...
3,2708,14606598,2014-06-23,2247288,Camilla,Charles is just amazing and he made my stay sp...,2014,None,Charles is just amazing and he made my stay sp...
4,2708,39597339,2015-07-25,27974696,Fallon,Staying with Chas was an absolute pleasure. He...,2015,None,Staying with Chas was an absolute pleasure. He...
...,...,...,...,...,...,...,...,...,...
1144100,53632414,510261007827810772,2021-12-04,178062505,Shamgar,it's right next to the mAll,2021,None,it's right next to the mAll
1144101,53632780,510214334939624899,2021-12-04,80937500,Ed,Great location and place to stay,2021,None,Great location and place to stay
1144102,53646927,509546507889721376,2021-12-03,182275253,Jesse,Tina’s place was extremely clean and spacious....,2021,None,Tina’s place was extremely clean and spacious....
1144103,53664855,510925217069714776,2021-12-05,163323605,Hyia,Meifang is so kind. Communication was great. E...,2021,None,Meifang is so kind. Communication was great. E...


In [16]:
# keep only reviews in English
# replace non-english comments with 'Not English'
def non_eng(x):
    try:
        if detect(x) != 'en':
            return 'Not English'
    except:
        pass
# create a new column for whether the comments are in English

la_reviews['is_english'] = la_reviews.comments.apply(non_eng)

In [23]:
la_reviews.is_english.value_counts()

Not English    66002
Name: is_english, dtype: int64

In [ ]:
la_reviews.is_english.unique()

In [ ]:
la_reviews.head()

In [24]:
la_reviews['city'] = 'LA'
la_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,year,is_english,comments_adj,city
0,109,449036,2011-08-15,927861,Edwin,The host canceled my reservation the day befor...,2011,None,The host canceled my reservation the day befor...,LA
1,109,74506539,2016-05-15,22509885,Jenn,Me and two friends stayed for four and a half ...,2016,None,Me and two friends stayed for four and a half ...,LA
2,2708,13994902,2014-06-09,10905424,Kuberan,i had a wonderful stay. Everything from start ...,2014,None,i had a wonderful stay. Everything from start ...,LA
3,2708,14606598,2014-06-23,2247288,Camilla,Charles is just amazing and he made my stay sp...,2014,None,Charles is just amazing and he made my stay sp...,LA
4,2708,39597339,2015-07-25,27974696,Fallon,Staying with Chas was an absolute pleasure. He...,2015,None,Staying with Chas was an absolute pleasure. He...,LA


In [25]:
# create a new dataframe with only reviews that in English
new_df2 = la_reviews[la_reviews['is_english'] != 'Not English']
new_df2.shape

(1076993, 10)

## Join the TWO cities

In [26]:
dfs = [new_df, new_df2]
all_reviews = pd.concat(dfs)
all_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1345387 entries, 0 to 1144103
Data columns (total 10 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   listing_id     1345387 non-null  int64         
 1   id             1345387 non-null  int64         
 2   date           1345387 non-null  datetime64[ns]
 3   reviewer_id    1345387 non-null  int64         
 4   reviewer_name  1345387 non-null  object        
 5   comments       1345387 non-null  object        
 6   year           1345387 non-null  int64         
 7   comments_adj   1345387 non-null  object        
 8   is_english     0 non-null        object        
 9   city           1345387 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 112.9+ MB


In [27]:
all_reviews.to_csv('English_ALL_SF_LA_reviews.csv', index=False)

In [28]:
#use only 2015-2021 data
df = all_reviews[ all_reviews['year'] >= 2015 ]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1296106 entries, 32 to 1144103
Data columns (total 10 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   listing_id     1296106 non-null  int64         
 1   id             1296106 non-null  int64         
 2   date           1296106 non-null  datetime64[ns]
 3   reviewer_id    1296106 non-null  int64         
 4   reviewer_name  1296106 non-null  object        
 5   comments       1296106 non-null  object        
 6   year           1296106 non-null  int64         
 7   comments_adj   1296106 non-null  object        
 8   is_english     0 non-null        object        
 9   city           1296106 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 108.8+ MB


In [29]:
df.to_csv('English_2015_2021_SF_LA_reviews.csv', index=False)